<a href="https://colab.research.google.com/github/satwik-sanka/pyspark-practice/blob/main/challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!rm -rf spark-3.5.1-bin-hadoop3 spark-2.3.1-bin-hadoop2.7 spark-3.3.2-bin-hadoop3
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark pyspark py4j

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

import findspark
findspark.init()


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,373 kB]
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Pack

In [3]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [4]:
mydata = spark.read.format("csv").option("header", "true").load("challenge.csv")

In [5]:
mydata.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

# Check if the country is Mexico - Add new y/n column

In [10]:
from pyspark.sql.functions import *

In [17]:
import pyspark.sql.functions as sqlfun

In [11]:
mydata = mydata.withColumn("from_mexico", when(mydata.Country == 'Mexico', lit("yes")).otherwise(lit("no")))

In [12]:
mydata.show()

+---------------+--------------+-----------------+----------+-----------+
|     ip_address|       Country|      Domain Name|Bytes_used|from_mexico|
+---------------+--------------+-----------------+----------+-----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|         no|
| 119.239.207.13|         China|         youtu.be|        51|         no|
|  68.69.217.210|         China|        adobe.com|        10|         no|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|         no|
|   211.13.10.68|     Indonesia|          hud.gov|        29|         no|
|   239.80.21.97|      Suriname|       smh.com.au|       218|         no|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|         no|
| 127.242.24.138|         China| surveymonkey.com|       123|         no|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|         no|
|   237.54.11.63|         China|       amazon.com|        83|         no|
| 252.141.157.25|         Japan|      

## Check the sum of bytes used from mexico vs non mexico

In [18]:
mydata.groupby("from_mexico").agg(sqlfun.sum(mydata.Bytes_used)).show()


+-----------+---------------+
|from_mexico|sum(Bytes_used)|
+-----------+---------------+
|         no|       508076.0|
|        yes|         6293.0|
+-----------+---------------+



## Group by country and get the unique no. of IP addresses within each country

In [21]:
d3 = mydata.groupBy("Country").agg(sqlfun.countDistinct(mydata.ip_address).alias('number_of_ips'))
d3.sort(col('number_of_ips').desc()).show()

+--------------+-------------+
|       Country|number_of_ips|
+--------------+-------------+
|         China|          172|
|     Indonesia|          114|
|   Philippines|           65|
|        Russia|           56|
|        Brazil|           35|
|        Poland|           31|
|        Sweden|           28|
|         Japan|           25|
|Czech Republic|           23|
|      Portugal|           23|
|        France|           21|
|          Peru|           19|
|      Colombia|           17|
| United States|           15|
|     Argentina|           14|
|       Ukraine|           14|
|        Mexico|           13|
|      Thailand|           12|
|       Nigeria|           11|
|        Canada|           11|
+--------------+-------------+
only showing top 20 rows

